In [1]:
%run ../load_files.ipynb

In [2]:
%run ../data_all_matrixes.ipynb

(98, 39)


In [3]:
%run ../y_multilevel.ipynb

,attention,memory
ID,,
FIS_001,0,0
FIS_002,0,0
FIS_004,0,0
FIS_005,0,0
FIS_007,1,0


In [4]:
y = y_values.memory

In [5]:
# Training and test subsamples

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.333, random_state=2018, stratify=y)

In [6]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(65, 39)
(33, 39)
(65,)
(33,)


In [7]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

from neudataload.transformer import CombineMatrixTransformer, SpreadOutMatrixTransformer, FeatureMatrixTransformer
from neudataload.utils import all_combinations

# Pipeline configuration
pipe = Pipeline([
    ('combining', CombineMatrixTransformer(column_name='combined')),
    ('spread_out', SpreadOutMatrixTransformer(columns=['combined'])),
    ('filter', FeatureMatrixTransformer(matrix_columns=['combined'])),

    ('reduce_dim', PCA()),
    ('classify', LogisticRegression(solver='liblinear'))
])

In [8]:
# Parametrized values
N_FEATURES_OPTIONS = range(1, 50, 3)
PENALTY = ['l1', 'l2']
REGULATION_C = np.linspace(1, 100, 90)
MATRIXES = list(all_combinations(column_matrixes))
EXTRA_1D_COLUMS = [None] # + list(all_combinations(available_columns))                            


param_grid = { 
        'combining__columns': MATRIXES,
        'filter__columns': EXTRA_1D_COLUMS,
        
        'reduce_dim__n_components': N_FEATURES_OPTIONS,
        'classify__penalty': PENALTY,
        'classify__C' : REGULATION_C,
}

In [9]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold

# Search the best parameter combination with Cross Validation 
cv = StratifiedKFold(n_splits=3, shuffle=True)

grid = RandomizedSearchCV(pipe, cv=cv, n_jobs=-1, param_distributions=param_grid, 
                          verbose=3,
                          scoring='accuracy',
                          n_iter = 150)


In [10]:
grid.fit(X_train, y_train)

Fitting 3 folds for each of 150 candidates, totalling 450 fits
[CV] reduce_dim__n_components=16, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD', 'DTI_RX'), classify__penalty=l2, classify__C=74.41573033707866 
[CV] reduce_dim__n_components=16, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD', 'DTI_RX'), classify__penalty=l2, classify__C=74.41573033707866 
[CV] reduce_dim__n_components=16, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD', 'DTI_RX'), classify__penalty=l2, classify__C=74.41573033707866 
[CV] reduce_dim__n_components=37, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD', 'RAW'), classify__penalty=l2, classify__C=57.73033707865169 
[CV] reduce_dim__n_components=37, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD', 'RAW'), classify__penalty=l2, classify__C=57.73033707865169 
[CV] reduce_dim__n_components=37, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD', 'RAW'), classify_

[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    7.6s


[CV]  reduce_dim__n_components=16, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'RAW', 'LS'), classify__penalty=l2, classify__C=47.71910112359551, score=0.5714285714285714, total=   2.4s
[CV] reduce_dim__n_components=4, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD', 'RAW'), classify__penalty=l1, classify__C=97.7752808988764 
[CV]  reduce_dim__n_components=13, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'LS'), classify__penalty=l1, classify__C=88.87640449438203, score=0.6818181818181818, total=   2.1s
[CV] reduce_dim__n_components=4, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD', 'RAW'), classify__penalty=l1, classify__C=97.7752808988764 
[CV]  reduce_dim__n_components=13, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'LS'), classify__penalty=l1, classify__C=88.87640449438203, score=0.4090909090909091, total=   2.3s
[CV] reduce_dim__n_components=4, filter__columns=None, combining__column

[CV] reduce_dim__n_components=25, filter__columns=None, combining__columns=('LS',), classify__penalty=l2, classify__C=92.21348314606742 
[CV]  reduce_dim__n_components=28, filter__columns=None, combining__columns=('RAW', 'LS'), classify__penalty=l2, classify__C=18.797752808988765, score=0.5454545454545454, total=   2.8s
[CV] reduce_dim__n_components=25, filter__columns=None, combining__columns=('LS',), classify__penalty=l2, classify__C=92.21348314606742 
[CV]  reduce_dim__n_components=28, filter__columns=None, combining__columns=('RAW', 'LS'), classify__penalty=l2, classify__C=18.797752808988765, score=0.5, total=   3.2s
[CV] reduce_dim__n_components=19, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD', 'RAW'), classify__penalty=l1, classify__C=44.38202247191011 
[CV]  reduce_dim__n_components=28, filter__columns=None, combining__columns=('RAW', 'LS'), classify__penalty=l2, classify__C=18.797752808988765, score=0.5238095238095238, total=   4.2s
[CV] reduce_dim__n_

[CV]  reduce_dim__n_components=34, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD', 'DTI_RX', 'LS'), classify__penalty=l1, classify__C=96.66292134831461, score=0.5, total=   3.7s
[CV] reduce_dim__n_components=28, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'RAW'), classify__penalty=l1, classify__C=38.82022471910113 
[CV]  reduce_dim__n_components=34, filter__columns=None, combining__columns=('DTI_FA', 'DTI_MD', 'DTI_RX', 'RAW'), classify__penalty=l1, classify__C=36.59550561797753, score=0.6818181818181818, total=   4.5s
[CV] reduce_dim__n_components=28, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'RAW'), classify__penalty=l1, classify__C=38.82022471910113 
[CV]  reduce_dim__n_components=34, filter__columns=None, combining__columns=('DTI_FA', 'DTI_MD', 'DTI_RX', 'RAW'), classify__penalty=l1, classify__C=36.59550561797753, score=0.5, total=   4.3s
[CV] reduce_dim__n_components=46, filter__columns=None, combining__columns=('DTI_L1

...

[CV] reduce_dim__n_components=25, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD', 'DTI_RX', 'RAW'), classify__penalty=l2, classify__C=96.66292134831461 
[CV]  reduce_dim__n_components=22, filter__columns=None, combining__columns=('DTI_FA', 'RAW', 'LS'), classify__penalty=l1, classify__C=31.03370786516854, score=0.5909090909090909, total=   3.1s
[CV] reduce_dim__n_components=37, filter__columns=None, combining__columns=('DTI_MD', 'LS'), classify__penalty=l2, classify__C=93.32584269662922 
[CV]  reduce_dim__n_components=22, filter__columns=None, combining__columns=('DTI_FA', 'RAW', 'LS'), classify__penalty=l1, classify__C=31.03370786516854, score=0.47619047619047616, total=   3.5s
[CV] reduce_dim__n_components=37, filter__columns=None, combining__columns=('DTI_MD', 'LS'), classify__penalty=l2, classify__C=93.32584269662922 
[CV]  reduce_dim__n_components=16, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_RX'), classify__penalty=l1, classify__C=47.7191011

[CV]  reduce_dim__n_components=37, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD', 'DTI_RX', 'LS'), classify__penalty=l2, classify__C=92.21348314606742, score=0.45454545454545453, total=   1.3s
[CV]  reduce_dim__n_components=7, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_RX', 'LS'), classify__penalty=l2, classify__C=24.359550561797754, score=0.5909090909090909, total=   2.1s
[CV]  reduce_dim__n_components=37, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD', 'DTI_RX', 'LS'), classify__penalty=l2, classify__C=92.21348314606742, score=0.7272727272727273, total=   1.4s
[CV] reduce_dim__n_components=1, filter__columns=None, combining__columns=('DTI_L1', 'DTI_RX'), classify__penalty=l1, classify__C=13.235955056179776 
[CV] reduce_dim__n_components=1, filter__columns=None, combining__columns=('DTI_L1', 'DTI_RX'), classify__penalty=l1, classify__C=13.235955056179776 
[CV] reduce_dim__n_components=1, filter__columns=None, com

[CV] reduce_dim__n_components=43, filter__columns=None, combining__columns=('DTI_L1',), classify__penalty=l2, classify__C=73.30337078651685 
[CV]  reduce_dim__n_components=1, filter__columns=None, combining__columns=('DTI_FA', 'DTI_MD', 'DTI_RX', 'LS'), classify__penalty=l1, classify__C=27.69662921348315, score=0.6818181818181818, total=   1.9s
[CV]  reduce_dim__n_components=1, filter__columns=None, combining__columns=('DTI_FA', 'DTI_MD', 'DTI_RX', 'LS'), classify__penalty=l1, classify__C=27.69662921348315, score=0.6363636363636364, total=   1.8s
[CV]  reduce_dim__n_components=1, filter__columns=None, combining__columns=('DTI_FA', 'DTI_MD', 'DTI_RX', 'LS'), classify__penalty=l1, classify__C=27.69662921348315, score=0.7619047619047619, total=   2.1s
[CV] reduce_dim__n_components=43, filter__columns=None, combining__columns=('DTI_L1',), classify__penalty=l2, classify__C=73.30337078651685 
[CV] reduce_dim__n_components=43, filter__columns=None, combining__columns=('DTI_L1',), classify__pe

[Parallel(n_jobs=-1)]: Done 450 out of 450 | elapsed:  3.2min finished


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=True),
          error_score='raise',
          estimator=Pipeline(memory=None,
     steps=[('combining', CombineMatrixTransformer(column_name='combined', columns=None,
             op=<function mean at 0x7fea14012268>)), ('spread_out', SpreadOutMatrixTransformer(columns=['combined'], symmetric=True)), ('filter', FeatureMatrixTransformer(columns=None, matrix_columns=['combined'])), ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
          fit_params=None, iid=True, n_iter=150, n_jobs=-1,
          param_distributions={'combining__columns': [('DTI_FA',), ('DTI_L1',), ('DTI_MD',), ('DTI_RX',), ('RAW',), ('LS',), ('DTI_FA', 'DTI_L1'), ('DTI_FA', 'DTI_MD'), ('DTI_FA', 'DTI_RX'), ('DTI_FA', 'RAW'), ('DTI_FA', 'LS'), ('DTI_L1', 'DTI_MD'), ('DTI_L1', 'DTI_RX'), ('DTI_L1', 'RAW'), ('DTI_L1', 'LS'), (...fy__penalty': ['l1', 'l2'], 'classify__C': array([  1.     

In [11]:
%run ../grid_results.ipynb

reduce_dim__n_components = 1
filter__columns = None
combining__columns = ('DTI_FA', 'DTI_L1', 'DTI_RX', 'LS')
classify__penalty = l1
classify__C = 37.70786516853933
Test
Accuracy in test: 72.73%
             precision    recall  f1-score   support

          0       0.70      1.00      0.82        21
          1       1.00      0.25      0.40        12

avg / total       0.81      0.73      0.67        33

Train
Accuracy in train: 67.69%
             precision    recall  f1-score   support

          0       0.68      0.95      0.79        42
          1       0.67      0.17      0.28        23

avg / total       0.67      0.68      0.61        65

